In [5]:
#Opcion Producto=68 . Lunes 1 de Julio esta es la que mejor decodifica de madrugda

import pandas as pd
import json
import numpy as np

# Paso 1: Cargar el dataset original
df = pd.read_csv('./datasets/item_features.csv', encoding='utf-8')

# Cargar el JSON con las categorías
with open('literales.json', 'r', encoding='utf-8') as f:
    categorias = json.load(f)

# JSON con características específicas de calzado
caracteristicas_calzado = {
    "tipos_de_suela_calzado": [
        "plana", "tacón", "plataforma", "cuña", "suela de goma", "suela de madera", "suela antideslizante", 
        "suela de corcho", "suela de esparto"
    ],
    "tipos_de_tacon_calzado": [
        "stiletto", "cuña", "bloque", "kitten", "aguja", "plataforma", "tacón ancho", "tacón bajo", "tacón alto"
    ],
    "tipos_de_puntera_calzado": [
        "redonda", "puntiaguda", "abierta", "cuadrada", "en almendra", "puntera reforzada", "puntera metálica", 
        "puntera descubierta", "puntera texturizada"
    ],
}

# Características específicas de accesorios
caracteristicas_accesorios = {
    "tipos_de_gafas": [
        "sol", "graduadas", "deportivas", "de lectura"
    ],
    "tipos_de_joyeria": [
        "pendientes", "anillos", "pulseras", "collares", "broches", "tobilleras", 
        "colgantes", "relojes", "gemelos"
    ],
    "tipos_de_bolsos": [
        "de mano", "bandolera", "mochila", "tote", "clutch", "hobo", "bolsa de playa", 
        "bolsa de viaje", "riñonera"
    ],
    "tipos_de_sombreros": [
        "gorra", "sombrero de ala", "boina", "fedora", "panamá", "cloché", "sombrero cowboy", 
        "boina francesa", "sombrero bombín"
    ],
    "tipos_de_corbatas": [
        "tradicional", "pajarita", "pañuelo de cuello", "corbata delgada", "ascot", 
        "corbata reversible", "corbata estampada", "corbata tejida", "corbata estrecha"
    ],
    "materiales": [
      "Plástico", "Acero inoxidable", "Plata", "Oro","Fibra de carbono", "PVC"
  ],
}

# Lista de tipos específicos de calzado
tipos_calzado = [
    "Zapatillas deportivas", "Sandalias", "Botas", "Zapatos de salón", "Zapatillas de casa", "Botas de agua",
    "Botas de montaña", "Zapatillas", "Bailarinas", "Mocasines", "Zuecos", "Zapatos de plataforma", "Zapatillas de Paseo",
    "Botines", "Sandalias de caballero"
]

# Lista de tipos específicos de accesorios
tipos_accesorios = [
    "Bolso bandolera", "Mochila", "Cartera", "Bufanda", "Guantes", "Pañuelo", "Gorra", 
    "Gafas de sol", "Reloj", "Pulsera", "Collar", "Sombrero", "Turbante", "Corbata", 
    "Calcetines", "Pendientes", "Pulsera de tobillo", "Diadema", "Anillo", "Cinturon", 
    "Tirantes", "Gemelos", "Medias"
]

# Paso 2: Definir el mapeo de valores para feature_category_id
mapeo_valores = {
    56: "rangos_de_precio",
    68: "productos",
    50: "talla",
    47: "materiales",
    61: "estaciones",
    72: "estampados",
    69: "detalles",
    7: "caracteristicas",
    55: "marca",
    30: "colores",
    4: "ocasion",
    32: "generos",
    19: "pais_de_fabricacion"
    
}


# Función para verificar si el producto es calzado
def es_calzado(row):
    producto = row['feature_category_id']
    nombre_producto = row['feature_value_id']
    return producto == 68 and nombre_producto in tipos_calzado

# Función para verificar si el producto es un accesorio
def es_accesorio(row):
    producto = row['feature_category_id']
    nombre_producto = row['feature_value_id']
    return producto == 68 and nombre_producto in tipos_accesorios

# Paso 3: Añadir la columna 'category_literal' basada en el mapeo
df['category_literal'] = df['feature_category_id'].map(mapeo_valores)

# Paso 4: Añadir la columna 'feature_literal' basada en las características específicas de calzado o accesorios
# Creamos un diccionario para almacenar el siguiente índice a asignar para cada categoría
indice_por_categoria = {cat: 0 for cat in categorias.keys()}
indice_por_categoria_calzado = {cat: 0 for cat in caracteristicas_calzado.keys()}
indice_por_categoria_accesorios = {cat: 0 for cat in caracteristicas_accesorios.keys()}

# Diccionario para almacenar las asignaciones previas de (feature_category_id, feature_value_id)
asignaciones_previas = {}

# Función para obtener el siguiente valor de la categoría
def obtener_siguiente_valor(categoria):
    valores = categorias[categoria]
    indice_actual = indice_por_categoria[categoria]
    valor = valores[indice_actual]
    indice_por_categoria[categoria] = (indice_actual + 1) % len(valores)  # Circular por la lista de valores
    return valor

# Función para obtener el siguiente valor de características específicas de calzado
def obtener_siguiente_valor_calzado(categoria):
    valores = caracteristicas_calzado.get(categoria, [])
    if valores:  # Verifica que la lista no esté vacía
        indice_actual = indice_por_categoria_calzado[categoria]
        valor = valores[indice_actual]
        indice_por_categoria_calzado[categoria] = (indice_actual + 1) % len(valores)  # Circular por la lista de valores
        return valor
    else:
        return obtener_siguiente_valor(categoria)

# Función para obtener el siguiente valor de características específicas de accesorios
def obtener_siguiente_valor_accesorios(categoria):
    valores = caracteristicas_accesorios.get(categoria, [])
    if valores:  # Verifica que la lista no esté vacía
        indice_actual = indice_por_categoria_accesorios[categoria]
        valor = valores[indice_actual]
        indice_por_categoria_accesorios[categoria] = (indice_actual + 1) % len(valores)  # Circular por la lista de valores
        return valor
    else:
        return obtener_siguiente_valor(categoria)

# Iterar sobre las filas y asignar 'feature_literal'
def asignar_feature_literal(row):
    feature_id = row['feature_category_id']
    feature_value_id = row['feature_value_id']
    categoria = row['category_literal']
    
    if pd.isna(categoria):  # Manejar el caso donde 'category_literal' es NaN
        return np.nan
    
    # Crear una clave para la combinación de feature_category_id y feature_value_id
    clave = (feature_id, feature_value_id)
    
    # Usar la asignación previa si existe
    if clave in asignaciones_previas:
        return asignaciones_previas[clave]

    # Asignar un nuevo valor
    if es_calzado(row):
        valor = obtener_siguiente_valor_calzado(categoria)
    elif es_accesorio(row):
        valor = obtener_siguiente_valor_accesorios(categoria)
    else:
        valor = obtener_siguiente_valor(categoria)
    
    # Guardar la asignación para futuros usos
    asignaciones_previas[clave] = valor
    
    return valor

df['feature_literal'] = df.apply(asignar_feature_literal, axis=1)

#Duplico id para analisis posterior en excel
df['bis'] = df['item_id']

# Paso 5: Guardar el DataFrame actualizado en un nuevo archivo CSV
df.to_csv('items3_conliterales.csv', index=False, encoding='utf-8')


In [1]:
#Opcion Producto=68 . Lunes 1 de Julio esta es la que mejor decodifica a las 10.30

import pandas as pd
import json
import numpy as np

# Paso 1: Cargar el dataset original
df = pd.read_csv('./datasets/item_features.csv', encoding='utf-8')

# Cargar el JSON con las categorías
with open('literales.json', 'r', encoding='utf-8') as f:
    categorias = json.load(f)

# JSON con características específicas de calzado
caracteristicas_calzado = {
    "tipos_de_suela_calzado": [
        "plana", "tacón", "plataforma", "cuña", "suela de goma", "suela de madera", "suela antideslizante", 
        "suela de corcho", "suela de esparto"
    ],
    "tipos_de_tacon_calzado": [
        "stiletto", "cuña", "bloque", "kitten", "aguja", "plataforma", "tacón ancho", "tacón bajo", "tacón alto"
    ],
    "tipos_de_puntera_calzado": [
        "redonda", "puntiaguda", "abierta", "cuadrada", "en almendra", "puntera reforzada", "puntera metálica", 
        "puntera descubierta", "puntera texturizada"
    ],
    "materiales": [
        "Piel sintética", "Cuero"
    ]
}

# Características específicas de accesorios
caracteristicas_accesorios = {
    "tipos_de_gafas": [
        "sol", "graduadas", "deportivas", "de lectura"
    ],
    "tipos_de_joyeria": [
        "pendientes", "anillos", "pulseras", "collares", "broches", "tobilleras", 
        "colgantes", "relojes", "gemelos"
    ],
    "tipos_de_bolsos": [
        "de mano", "bandolera", "mochila", "tote", "clutch", "hobo", "bolsa de playa", 
        "bolsa de viaje", "riñonera"
    ],
    "tipos_de_sombreros": [
        "gorra", "sombrero de ala", "boina", "fedora", "panamá", "cloché", "sombrero cowboy", 
        "boina francesa", "sombrero bombín"
    ],
    "tipos_de_corbatas": [
        "tradicional", "pajarita", "pañuelo de cuello", "corbata delgada", "ascot", 
        "corbata reversible", "corbata estampada", "corbata tejida", "corbata estrecha"
    ],
    "materiales": [
        "Plástico", "Acero inoxidable", "Plata", "Oro", "Fibra de carbono", "PVC"
    ]
}

# Lista de tipos específicos de calzado
tipos_calzado = [
    "Zapatillas deportivas", "Sandalias", "Botas", "Zapatos de salón", "Zapatillas de casa", "Botas de agua",
    "Botas de montaña", "Zapatillas", "Bailarinas", "Mocasines", "Zuecos", "Zapatos de plataforma", "Zapatillas de Paseo",
    "Botines", "Sandalias de caballero"
]

# Lista de tipos específicos de accesorios
tipos_accesorios = [
    "Bolso bandolera", "Mochila", "Cartera", "Bufanda", "Guantes", "Pañuelo", "Gorra", 
    "Gafas de sol", "Reloj", "Pulsera", "Collar", "Sombrero", "Turbante", "Corbata", 
    "Calcetines", "Pendientes", "Pulsera de tobillo", "Diadema", "Anillo", "Cinturon", 
    "Tirantes", "Gemelos", "Medias"
]

# Paso 2: Definir el mapeo de valores para feature_category_id
mapeo_valores = {
    56: "rangos_de_precio",
    68: "productos",
    50: "talla",
    47: "materiales",
    61: "estaciones",
    72: "estampados",
    69: "detalles",
    7: "caracteristicas",
    55: "marca",
    30: "colores",
    4: "ocasion",
    32: "generos",
    19: "pais_de_fabricacion"
}

# Función para verificar si el producto es calzado
def es_calzado(row):
    producto = row['feature_category_id']
    nombre_producto = row['feature_value_id']
    return producto == 68 and nombre_producto in tipos_calzado

# Función para verificar si el producto es un accesorio
def es_accesorio(row):
    producto = row['feature_category_id']
    nombre_producto = row['feature_value_id']
    return producto == 68 and nombre_producto in tipos_accesorios

# Paso 3: Añadir la columna 'category_literal' basada en el mapeo
df['category_literal'] = df['feature_category_id'].map(mapeo_valores)

# Paso 4: Añadir la columna 'feature_literal' basada en las características específicas de calzado o accesorios
# Creamos un diccionario para almacenar el siguiente índice a asignar para cada categoría
indice_por_categoria = {cat: 0 for cat in categorias.keys()}
indice_por_categoria_calzado = {cat: 0 for cat in caracteristicas_calzado.keys()}
indice_por_categoria_accesorios = {cat: 0 for cat in caracteristicas_accesorios.keys()}

# Diccionario para almacenar las asignaciones previas de (feature_category_id, feature_value_id)
asignaciones_previas = {}

# Función para obtener el siguiente valor de la categoría
def obtener_siguiente_valor(categoria):
    valores = categorias[categoria]
    indice_actual = indice_por_categoria[categoria]
    valor = valores[indice_actual]
    indice_por_categoria[categoria] = (indice_actual + 1) % len(valores)  # Circular por la lista de valores
    return valor

# Función para obtener el siguiente valor de características específicas de calzado
def obtener_siguiente_valor_calzado(categoria):
    valores = caracteristicas_calzado.get(categoria, [])
    if valores:  # Verifica que la lista no esté vacía
        indice_actual = indice_por_categoria_calzado[categoria]
        valor = valores[indice_actual]
        indice_por_categoria_calzado[categoria] = (indice_actual + 1) % len(valores)  # Circular por la lista de valores
        return valor
    else:
        return obtener_siguiente_valor(categoria)

# Función para obtener el siguiente valor de características específicas de accesorios
def obtener_siguiente_valor_accesorios(categoria):
    valores = caracteristicas_accesorios.get(categoria, [])
    if valores:  # Verifica que la lista no esté vacía
        indice_actual = indice_por_categoria_accesorios[categoria]
        valor = valores[indice_actual]
        indice_por_categoria_accesorios[categoria] = (indice_actual + 1) % len(valores)  # Circular por la lista de valores
        return valor
    else:
        return obtener_siguiente_valor(categoria)

# Iterar sobre las filas y asignar 'feature_literal'
def asignar_feature_literal(row):
    feature_id = row['feature_category_id']
    feature_value_id = row['feature_value_id']
    categoria = row['feature_category_literal']
    
    if pd.isna(categoria):  # Manejar el caso donde 'category_literal' es NaN
        return np.nan
    
    # Crear una clave para la combinación de feature_category_id y feature_value_id
    clave = (feature_id, feature_value_id)
    
    # Usar la asignación previa si existe
    if clave in asignaciones_previas:
        return asignaciones_previas[clave]

    # Asignar un nuevo valor
    if es_calzado(row):
        if categoria == 'materiales':
            valor = obtener_siguiente_valor_calzado(categoria)
        else:
            valor = obtener_siguiente_valor_calzado(categoria)
    elif es_accesorio(row):
        if categoria == 'materiales':
            valor = obtener_siguiente_valor_accesorios(categoria)
        else:
            valor = obtener_siguiente_valor_accesorios(categoria)
    else:
        valor = obtener_siguiente_valor(categoria)
    
    # Guardar la asignación para futuros usos
    asignaciones_previas[clave] = valor
    
    return valor

df['feature_value_literal'] = df.apply(asignar_feature_literal, axis=1)

# Duplicar id para análisis posterior en excel
df['item_id_2'] = df['item_id']

# Paso 5: Guardar el DataFrame actualizado en un nuevo archivo CSV
df.to_csv('./item_features_semirelleno_con_literales.csv', index=False, encoding='utf-8')



### Comprabaciones y chequeos previos al cuadre de ... 
### literales(feature_category_literal/feature_value_literal) y identificadores (feature_category_id/feature_value_id)

In [7]:
import pandas as pd

# Cargar el fichero CSV
df = pd.read_csv('./item_features_semirelleno_con_literales.csv')

# Filtrar filas donde el feature_category_literal no es nulo
assigned_literals = df[df['feature_category_literal'].notna()]

# Agrupar por feature_category_id y obtener los literales únicos asociados
category_analysis = assigned_literals.groupby('feature_category_id')['feature_category_literal'].unique()

# Convertir el resultado a un diccionario para su análisis
category_analysis_dict = category_analysis #.to_dict()

# Obtener todos los feature_category_id presentes en el CSV
all_category_ids = df['feature_category_id'].unique()

# Identificar los feature_category_id que no tienen literal asignado
unassigned_category_ids = set(all_category_ids) - set(category_analysis_dict.keys())

# Mostrar los resultados
print("Asignados:")
print(category_analysis_dict)
print("\nNo asignados:")
print(unassigned_category_ids)

# Opcionalmente, guardar los resultados en variables para su uso posterior
assigned_dict = category_analysis_dict
unassigned_list = list(unassigned_category_ids)



Asignados:
feature_category_id
4                 [ocasion]
7         [caracteristicas]
19    [pais_de_fabricacion]
30                [colores]
32                [generos]
47             [materiales]
50                  [talla]
55                  [marca]
56       [rangos_de_precio]
61             [estaciones]
68              [productos]
69               [detalles]
72             [estampados]
Name: feature_category_literal, dtype: object

No asignados:
{1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 51, 52, 53, 54, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 70, 71, 73}


In [6]:
import json

# Cargar el fichero JSON
with open('./literales.json', 'r', encoding='utf-8') as f:
    category_data = json.load(f)

# Obtener los nombres de las categorías (las claves principales del JSON)
category_literals = list(category_data.keys())

# Mostrar los literales de las categorías
print(category_literals)

# Opcionalmente, guardar los literales en una variable para su uso posterior
category_literals_list = category_literals


['materiales', 'talla', 'productos', 'generos', 'estaciones', 'colores', 'ocasion', 'caracteristicas', 'estampados', 'detalles', 'tipos_de_corte', 'tipos_de_cuello', 'tipos_de_cintura', 'tipos_de_manga', 'tipos_de_cierre', 'tipos_de_bolsillos', 'tipos_de_suela_calzado', 'tipos_de_tacon_calzado', 'tipos_de_puntera_calzado', 'tipos_de_gafas', 'tipos_de_joyeria', 'tipos_de_bolsos', 'tipos_de_sombreros', 'tipos_de_corbatas', 'material_del_forro', 'material_del_relleno', 'transpirabilidad', 'impermeabilidad', 'durabilidad', 'facilidad_de_cuidado', 'resistencia_a_las_arrugas', 'elasticidad', 'suavidad', 'peso', 'calidez', 'resistencia_a_la_decoloracion', 'resistencia_al_encogimiento', 'resistencia_a_la_pilling', 'resistencia_a_la_abrasion', 'resistencia_a_los_rayos_uv', 'resistencia_al_fuego', 'resistencia_a_las_manchas', 'resistencia_al_agua', 'resistencia_al_viento', 'resistencia_a_los_olores', 'resistencia_a_los_insectos', 'resistencia_a_los_microbios', 'biodegradabilidad', 'reciclabilida

In [8]:
import pandas as pd
import json

# Cargar el fichero JSON
with open('./literales.json', 'r', encoding='utf-8') as f:
    category_data = json.load(f)

# Obtener los nombres de las categorías del JSON
category_literals_json = set(category_data.keys())

# Cargar el fichero CSV
df = pd.read_csv('./item_features_semirelleno_con_literales.csv')

# Obtener los literales ya asignados en el CSV
assigned_literals_csv = set(df['feature_category_literal'].dropna().unique())

# Determinar los literales asignados y no asignados
assigned_literals = category_literals_json.intersection(assigned_literals_csv)
unassigned_literals = category_literals_json - assigned_literals_csv

# Mostrar los resultados
print("Literales asignados:")
print(assigned_literals)
print("\nLiterales no asignados:")
print(unassigned_literals)

# Opcionalmente, guardar los resultados en variables para su uso posterior
assigned_literals_set = assigned_literals
unassigned_literals_set = unassigned_literals


Literales asignados:
{'caracteristicas', 'talla', 'generos', 'materiales', 'detalles', 'marca', 'pais_de_fabricacion', 'estaciones', 'rangos_de_precio', 'productos', 'estampados', 'colores', 'ocasion'}

Literales no asignados:
{'durabilidad', 'resistencia_a_los_rayos_uv', 'certificaciones_comercio_justo', 'peso', 'tipo_de_tela', 'resistencia_al_viento', 'certificaciones_ecologicas', 'tipos_de_corte', 'tipo_de_ajuste_calzado', 'tipos_de_corbatas', 'tipos_de_cuello', 'tipo_de_montura_gafas', 'resistencia_al_encogimiento', 'tipos_de_cierre', 'suavidad', 'certificaciones_condiciones_laborales', 'opciones_de_personalizacion', 'tipos_de_bolsos', 'resistencia_a_los_microbios', 'instrucciones_de_cuidado', 'disponibilidad', 'garantia', 'tipo_de_cierre_joyeria', 'tipos_de_suela_calzado', 'tipo_de_asa_bolsos', 'material_del_forro', 'resistencia_al_agua', 'resistencia_a_la_decoloracion', 'tipos_de_joyeria', 'biodegradabilidad', 'Marca Nacional', 'elasticidad', 'resistencia_a_las_arrugas', 'certifi

### Análisis de literales e identificadores no asignados (19/ago/2024)

In [34]:
import pandas as pd

# Cargar el fichero CSV
# -item_features_semirelleno_con_literales.csv es el fichero original-
df = pd.read_csv('./item_features_semirelleno_con_literales_actualizado.csv')

# Obtener las combinaciones distintas de feature_category_id y feature_value_id
distinct_combinations = df[['feature_category_id', 'feature_value_id']].drop_duplicates()

# Mostrar las combinaciones distintas
#print("Combinaciones distintas de feature_category_id y feature_value_id:")
#print(distinct_combinations)

# Opcionalmente, contar el número de combinaciones distintas
num_combinations = distinct_combinations.shape[0]
print(f"\n(csv)Número total de combinaciones distintas: {num_combinations}")


# Filtrar filas donde el feature_category_literal no es nulo
assigned_literals = df[df['feature_category_literal'].notna()]

# Agrupar por feature_category_id y obtener los literales únicos asociados
category_analysis = assigned_literals.groupby('feature_category_id')['feature_category_literal'].unique()
print(f"(csv)Número de identificadores de categorías asociados ya {len(category_analysis)}")

# Convertir el resultado a un diccionario para su análisis
category_analysis_dict = category_analysis.to_dict()

# Obtener todos los feature_category_id presentes en el CSV
all_category_ids = df['feature_category_id'].unique()
print(f"(csv)Número de todas los categorias {len(all_category_ids)}")

# Identificar los feature_category_id que no tienen literal asignado
unassigned_category_ids = set(all_category_ids) - set(category_analysis_dict.keys())

# Filtrar filas donde el feature_value_id está vacío y el feature_category_id está en la lista de no asignados
empty_value_df = df[(df['feature_value_literal'].isna()) & (df['feature_category_id'].isin(unassigned_category_ids))]

# Agrupar por feature_category_id y recoger los feature_value_id correspondientes que están vacíos
unassigned_category_values_dict = empty_value_df.groupby('feature_category_id')['feature_value_id'].unique().to_dict()

# Ordenar el diccionario por el tamaño del array de feature_value_id's de mayor a menor
sorted_unassigned_category_values = dict(sorted(unassigned_category_values_dict.items(), key=lambda item: len(item[1]), reverse=True))

# Mostrar los resultados
#print("Feature Category IDs sin Literal Asignado ordenados por la cantidad de Feature Values Vacíos:")
#print(sorted_unassigned_category_values)

print(f"(csv)Número de identificadores de categorías sin asignar {len(sorted_unassigned_category_values)}")


print("--------------------------------------------------------------")

# Contar combinaciones distintas de feature_category_id y feature_value_id para sorted_unassigned_category_values
unassigned_combination_counts = {}
total1 = 0
for category_id, value_ids in sorted_unassigned_category_values.items():
    count = distinct_combinations[(distinct_combinations['feature_category_id'] == category_id) & 
                                  (distinct_combinations['feature_value_id'].isin(value_ids))].shape[0]
    unassigned_combination_counts[category_id] = count
    total1 += count
print(f"Total de combinaciones no asignadas: {total1}")
    
# Contar combinaciones distintas de feature_category_id y feature_value_id para category_analysis
assigned_combination_counts = {}
total2 = 0
for category_id in category_analysis_dict.keys():
    count = distinct_combinations[distinct_combinations['feature_category_id'] == category_id].shape[0]
    assigned_combination_counts[category_id] = count
    total2 += count
print(f"Total de combinaciones asignadas: {total2}")
print (f"Total: {total1+total2}")    
# Mostrar los resultados
#print("Feature Category IDs sin Literal Asignado ordenados por la cantidad de Feature Values Vacíos y combinaciones distintas:")
#for category_id, value_ids in sorted_unassigned_category_values.items():
#    print(f"Category ID: {category_id}, Feature Values Vacíos: {len(value_ids)}, Combinaciones Distintas: {unassigned_combination_counts[category_id]}")

#print(f"\nNúmero total de identificadores de categorías sin asignar: {len(sorted_unassigned_category_values)}")

#print("\nNúmero de combinaciones distintas para cada feature_category_id en category_analysis:")
#for category_id, count in assigned_combination_counts.items():
#    print(f"Category ID: {category_id}, Combinaciones Distintas: {count}")
#unassigned_combination_counts
assigned_combination_counts


(csv)Número total de combinaciones distintas: 904
(csv)Número de identificadores de categorías asociados ya 13
(csv)Número de todas los categorias 73
(csv)Número de identificadores de categorías sin asignar 60
--------------------------------------------------------------
Total de combinaciones no asignadas: 487
Total de combinaciones asignadas: 417
Total: 904


{4: 16,
 7: 35,
 19: 30,
 30: 67,
 32: 5,
 47: 17,
 50: 24,
 55: 49,
 56: 67,
 61: 5,
 68: 48,
 69: 29,
 72: 25}

In [41]:
import pandas as pd
import json

# Cargar el fichero JSON
#with open('./literales_fbc.json', 'r', encoding='utf-8') as f:
with open('./literales_ultimate.json', 'r', encoding='utf-8') as f:
    category_data = json.load(f)
    
# Obtener los nombres de las categorías del JSON
category_literals_json = set(category_data.keys())

print(f"(json)Número de categorías de literales totales {len(category_literals_json)}")

# Crear una lista para almacenar las combinaciones únicas
distinct_combinations = []

# Recorrer cada feature_category_id en el JSON
total = 0
for category_id, values in category_data.items():
    # Añadir cada par (feature_category_id, feature_value_id) a la lista
    for value_id in values:
        total +=1
        distinct_combinations.append((category_id, value_id))
        #print(f"({category_id},{value_id}) : {total} vs {len(list(set(distinct_combinations)))}")
#print(f"\n(json)Total contadas: {total}")

# Eliminar duplicados convirtiendo la lista a un conjunto, y luego de nuevo a lista
distinct_combinations = list(set(distinct_combinations))

# Mostrar las combinaciones distintas
#print("Combinaciones distintas de feature_category_id y feature_value_id:")
#print(distinct_combinations)

# Contar el número de combinaciones distintas
num_combinations = len(distinct_combinations)
print(f"\n(json)Número total de combinaciones distintas: {num_combinations}")
print("--------------------------------------------------------------")

# Cargar el fichero CSV
df = pd.read_csv('./item_features_semirelleno_con_literales_actualizado.csv')

# Obtener los literales ya asignados en el CSV
assigned_literals_csv = set(df['feature_category_literal'].dropna().unique())
print(f"Número de categorías de literales totales asignadas(csv) {len(assigned_literals_csv)}")

# Determinar los literales asignados y no asignados
assigned_literals = category_literals_json.intersection(assigned_literals_csv)
print(f"Número de categorías de literales totales asignadas(json) {len(assigned_literals)}")

unassigned_literals = category_literals_json - assigned_literals_csv
print(f"Número de categorías de literales totales sin asignar(json) {len(unassigned_literals)}")
print("--------------------------------------------------------------")

# Añadir el array de valores posibles para los literales no asignados
unassigned_literals_with_values = {literal: category_data[literal] for literal in unassigned_literals}
assigned_literals_with_values = {literal: category_data[literal] for literal in assigned_literals}

# Ordenar de mayor a menor según el número de elementos en el array de valores
unassigned_literals_sorted = dict(sorted(unassigned_literals_with_values.items(), key=lambda item: len(item[1]), reverse=True))
assigned_literals_sorted = dict(sorted(assigned_literals_with_values.items(), key=lambda item: len(item[1]), reverse=True))


print("---------------------------------------------------------------------------------------------")
print("Literales asignados con posibles valores, ordenados de mayor a menor por cantidad de valores:")
total = 0
for literal, values in assigned_literals_sorted.items():
    total += len(values)
    print(f"({len(values)}):{literal}:{values}")
print(f"Total {total} ------------------------------------------------------------------------------")

# Mostrar los resultados ordenados
print("Literales no asignados con posibles valores, ordenados de mayor a menor por cantidad de valores:")
total = 0
for literal, values in unassigned_literals_sorted.items():
    total += len(values)
    print(f"({len(values)}):{literal}:{values}")
print(f"Total {total} ------------------------------------------------------------------------------")    

print("--------------------------------------------------------------")
print(f"Número de combinaciones distintas no asignadas {total}")
#unassigned_literals_with_values_sorted_dict = unassigned_literals_sorted
print(f"Número de literales/categorías sin asignar {len(unassigned_literals_sorted)}")
print("--------------------------------------------------------------")

# Filtrar las combinaciones distintas en el CSV para las categorías asignadas
assigned_combinations_csv = df[df['feature_category_literal'].notna()][['feature_category_id', 'feature_value_id']].drop_duplicates()
assigned_combinations_count_csv = assigned_combinations_csv.shape[0]
print(f"Número total de combinaciones distintas en CSV para categorías asignadas: {assigned_combinations_count_csv}")

# Filtrar las combinaciones distintas desde el JSON para las categorías asignadas
assigned_combinations_json = [(category_id, value_id) for category_id, values in category_data.items() if category_id in assigned_literals for value_id in values]
assigned_combinations_json = list(set(assigned_combinations_json))
assigned_combinations_count_json = len(assigned_combinations_json)
print(f"Número total de combinaciones distintas en JSON para categorías asignadas: {assigned_combinations_count_json}")

(json)Número de categorías de literales totales 73

(json)Número total de combinaciones distintas: 904
--------------------------------------------------------------
Número de categorías de literales totales asignadas(csv) 13
Número de categorías de literales totales asignadas(json) 13
Número de categorías de literales totales sin asignar(json) 60
--------------------------------------------------------------
---------------------------------------------------------------------------------------------
Literales asignados con posibles valores, ordenados de mayor a menor por cantidad de valores:
(67):colores:['Azul', 'Blanca', 'Blanco', 'Negro', 'Rojo', 'Azul oscuro', 'Verde', 'Marrón', 'Gris', 'Rosa', 'Amarillo', 'Beige', 'Multicolor', 'Dorado', 'Plateado', 'Morado', 'Cyan', 'Antracita', 'Turquesa', 'Naranja', 'Verde lima', 'Granate', 'Índigo', 'Salmón', 'Caqui', 'Verde oliva', 'Malva', 'Lila', 'Fucsia', 'Coral', 'Terracota', 'Burdeos', 'Mostaza', 'Amarillo mostaza', 'Ocre', 'Aqua', 'Az

In [44]:
combinaciones_categorias_asignadas = df[df['feature_category_literal'].notna()][['feature_category_id','feature_category_literal']].drop_duplicates()
combinaciones_categorias_asignadas = combinaciones_categorias_asignadas.sort_values(by=['feature_category_id'])
print(combinaciones_categorias_asignadas)

combinaciones_asignadas = df[df['feature_category_literal'].notna()][['feature_category_id', 'feature_category_literal','feature_value_id', 'feature_value_literal']].drop_duplicates()

# Ordenar por feature_category_id y feature_value_id
combinaciones_asignadas_sorted = combinaciones_asignadas.sort_values(by=['feature_category_id', 'feature_value_id'])

df2 = combinaciones_asignadas_sorted

df_x = df2[df2['feature_category_id']==7]
print(f"Len(x) {len(df_x)}")

#repeated_values = df_x['feature_value_literal'].value_counts()
#repeated_values = repeated_values[repeated_values > 1]
#repeated_values
#valores_determinados = ['450-500'] 
#resultados = df_x[df_x['feature_value_literal'].isin(valores_determinados)]
#resultados

df_x.sort_values(by=['feature_value_literal', 'feature_value_literal'])
#print(df_x[30:])


     feature_category_id feature_category_literal
29                     4                  ocasion
10                     7          caracteristicas
21                    19      pais_de_fabricacion
129                   30                  colores
17                    32                  generos
12                    47               materiales
7                     50                    talla
25                    55                    marca
0                     56         rangos_de_precio
8                     61               estaciones
2                     68                productos
11                    69                 detalles
4                     72               estampados
Len(x) 35


,feature_category_id,feature_category_literal,feature_value_id,feature_value_literal
11812,7,caracteristicas,221,Absorbente
23100,7,caracteristicas,463,Acolchado
5513,7,caracteristicas,528,Adaptable
274,7,caracteristicas,619,Ajustable
1022,7,caracteristicas,28,Alta tracción
675,7,caracteristicas,2,Amortiguada
313,7,caracteristicas,794,Analógico
2624,7,caracteristicas,298,Antideslizante
369724,7,caracteristicas,710,Antiestático
3104,7,caracteristicas,492,Comodo


### Hecho, corregido en el fichero -item_features_semirelleno_con_literales_actualizado.csv-, ya no hace falta hacerlo de nuevo!

In [2]:
def update_literals(df, feature_category_id, feature_value_id, new_category_literal, new_value_literal):
    """
    Actualiza las ocurrencias de feature_category_literal y feature_value_literal en el DataFrame
    basadas en los valores proporcionados para feature_category_id y feature_value_id.

    Args:
    - df (pd.DataFrame): DataFrame que contiene las columnas a actualizar.
    - feature_category_id (str): El valor de feature_category_id para filtrar las filas.
    - feature_value_id (str): El valor de feature_value_id para filtrar las filas.
    - new_category_literal (str): El nuevo valor para feature_category_literal.
    - new_value_literal (str): El nuevo valor para feature_value_literal.

    Returns:
    - pd.DataFrame: DataFrame con los valores actualizados.
    """
    # Asegurarse de que las columnas existen en el DataFrame
    required_columns = ['feature_category_id', 'feature_value_id', 'feature_category_literal', 'feature_value_literal']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"El DataFrame debe contener la columna '{col}'")
    
    # Convertir los valores de entrada a numéricos si no lo son
    feature_category_id = pd.to_numeric(feature_category_id, errors='coerce')
    feature_value_id = pd.to_numeric(feature_value_id, errors='coerce')
    
    # Actualizar los valores en el DataFrame
    df.loc[
        (df['feature_category_id'] == feature_category_id) & 
        (df['feature_value_id'] == feature_value_id), 
        ['feature_category_literal', 'feature_value_literal']
    ] = new_category_literal, new_value_literal
    
    return df

In [111]:
marcas = ['Zara', 'Mango', 'Massimo Dutti', 'Pull&Bear', 'Bershka', 'Stradivarius', 'Desigual', 
          'El Ganso', 'Adolfo Domínguez', 'Camper', 'Pikolinos', 'Gioseppo', 'Castañer', 
          'Panama Jack', 'Mustang', 'Desigual', 'H&M', 'Uniqlo', 'Nike', 'Adidas', 'Puma', 
          'New Balance', 'Reebok', 'Converse', 'Vans', 'Dr. Martens', 'Timberland', 
          'Skechers', 'Clarks', 'Crocs', 'Tommy Hilfiger', 'Calvin Klein', 'Ralph Lauren', 
          "Levi's", 'Diesel', 'Guess', 'Michael Kors', 'Coach', 'Fossil', 'Kate Spade', 
          'Gucci', 'Prada', 'Louis Vuitton', 'Chanel', 'Burberry', 'Balenciaga', 'Valentino', 
          'Dior', 'Hermès', 'Givenchy', 'Saint Laurent', 'Versace', 'Jimmy Choo', 'Louboutin', 
          'Alexander McQueen', 'Bottega Veneta', 'Fendi', 'Moschino', 'Kenzo', 'Paul Smith', 
          'Tory Burch', 'Marc Jacobs', 'Swarovski', 'Armani', 'Max Mara', 'Miu Miu', 'Moncler', 
          'Salvatore Ferragamo', 'Stella McCartney', 'Thom Browne', "Tod's", 'Yves Saint Laurent', 
          'Zegna', 'Brunello Cucinelli', 'Rag & Bone', 'Acne Studios', 'Isabel Marant', 'Jacquemus', 
          'Off-White', 'Rick Owens', 'Balmain', 'Emilio Pucci', 'Loewe', 'Missoni', 
          'Dries Van Noten', 'Margiela', 'Erdem', 'Peter Pilotto', 'Simone Rocha', 'Vetements', 
          'Virgil Abloh', 'A-Cold-Wall*', 'Alyx', 'Fear of God']

# Ordenar la lista alfabéticamente
marcas_ordenadas = sorted(marcas)

df = update_literals(
    df, 
    feature_category_id='55', 
    feature_value_id='692', 
    new_category_literal='marca', 
    new_value_literal='Givenchy'
)

marcas_ordenadas

df = update_literals(
    df, 
    feature_category_id='69', 
    feature_value_id='260', 
    new_category_literal='detalles', 
    new_value_literal='Cremallera'
)


df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='436', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='500-550'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='866', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='550-600'
)


df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='344', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='400-410'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='460', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='410-420'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='523', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='420-430'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='844', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='430-450'
)




df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='355', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='350-360'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='400', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='360-370'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='606', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='370-380'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='868', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='380-400'
)




df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='194', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='300-310'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='402', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='310-320'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='626', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='320-330'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='843', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='330-350'
)




df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='89', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='250-260'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='164', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='260-270'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='503', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='270-280'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='541', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='280-300'
)




df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='19', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='200-210'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='23', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='210-220'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='334', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='220-230'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='496', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='230-250'
)



df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='29', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='150-160'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='161', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='160-170'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='439', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='170-180'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='589', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='180-200'
)



df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='50', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='100-110'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='74', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='110-120'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='332', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='120-130'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='749', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='130-150'
)


df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='102', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='90-92'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='176', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='92-94'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='192', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='94-96'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='527', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='96-100'
)


df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='1', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='80-82'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='408', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='82-84'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='792', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='84-86'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='804', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='86-90'
)




df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='32', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='70-72'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='131', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='72-74'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='249', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='74-76'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='506', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='76-80'
)



df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='231', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='60-62'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='418', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='62-64'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='450', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='64-66'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='501', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='66-70'
)


df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='239', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='50-52'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='295', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='52-54'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='581', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='54-56'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='757', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='56-60'
)



df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='223', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='40-42'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='429', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='42-44'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='594', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='44-46'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='752', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='46-50'
)



df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='191', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='30-32'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='427', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='32-34'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='787', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='34-36'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='791', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='36-40'
)



df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='39', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='20-22'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='153', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='22-24'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='337', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='24-26'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='741', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='26-30'
)


df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='312', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='10-12'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='365', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='12-14'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='761', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='14-16'
)
df = update_literals(
    df, 
    feature_category_id='56', 
    feature_value_id='819', 
    new_category_literal='rangos_de_precio', 
    new_value_literal='16-20'
)


























df = update_literals(
    df, 
    feature_category_id='30', 
    feature_value_id='482', 
    new_category_literal='colores', 
    new_value_literal='Púrpura'
)
df = update_literals(
    df, 
    feature_category_id='30', 
    feature_value_id='564', 
    new_category_literal='colores', 
    new_value_literal='Beige arena'
)
df = update_literals(
    df, 
    feature_category_id='30', 
    feature_value_id='809', 
    new_category_literal='colores', 
    new_value_literal='Turquesa claro'
)

df = update_literals(
    df, 
    feature_category_id='4', 
    feature_value_id='331', 
    new_category_literal='ocasion', 
    new_value_literal='Eventos Deportivos'
)
df = update_literals(
    df, 
    feature_category_id='4', 
    feature_value_id='385', 
    new_category_literal='ocasion', 
    new_value_literal='Cita'
)
df = update_literals(
    df, 
    feature_category_id='4', 
    feature_value_id='635', 
    new_category_literal='ocasion', 
    new_value_literal='Gala'
)
df = update_literals(
    df, 
    feature_category_id='4', 
    feature_value_id='865', 
    new_category_literal='ocasion', 
    new_value_literal='Temporada Festiva'
)

df = update_literals(
    df, 
    feature_category_id='7', 
    feature_value_id='838', 
    new_category_literal='caracteristicas',
    new_value_literal='Translucido'
)
df = update_literals(
    df, 
    feature_category_id='7', 
    feature_value_id='710', 
    new_category_literal='caracteristicas',
    new_value_literal='Antiestático'
)

In [43]:
# Guardar el DataFrame actualizado, si es necesario
# el original se saca de -item_features_semirelleno_con_literales.csv-
df.to_csv('./item_features_semirelleno_con_literales_actualizado.csv', index=False)

### Experimentos para asignar las categorias faltantes, mediante una estrategia avanzada, modelos de IA
### (20/ago/2024)

In [45]:
# (0) Experimento inicial 

# Resultado :: EL problema de esta solución, es que siempre recomienda el mismo literal a la categoría faltante!
# Verificación del Modelo de Imputación: Asegúrate de que el modelo de RandomForest está correctamente entrenado y validado. 
# Si el modelo está sobreajustado o no tiene suficiente variabilidad en los datos de entrenamiento, podría estar produciendo 
# resultados repetitivos.

import pandas as pd
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# Configurar tqdm para que se muestre una barra de progreso en las llamadas a pandas
tqdm.pandas()

# Cargar el fichero CSV
df = pd.read_csv('./item_features_semirelleno_con_literales_actualizado.csv')

# Cargar el fichero JSON
with open('./literales_ultimate.json', 'r', encoding='utf-8') as f:
    category_data = json.load(f)

# Preparar el DataFrame
print("Preparando el DataFrame...")
df['feature_category_id'] = df['feature_category_id'].astype('category')
df['feature_value_id'] = df['feature_value_id'].astype('category')
df['feature_category_literal'] = df['feature_category_literal'].astype('category')
df['feature_value_literal'] = df['feature_value_literal'].astype('category')

# Convertir JSON a DataFrame
category_literals = {category: set(literals) for category, literals in category_data.items()}

# Separar datos con y sin valores faltantes
print("Separando datos con y sin valores faltantes...")
data_with_values = df.dropna(subset=['feature_category_literal', 'feature_value_literal'])
data_missing_values = df[df['feature_category_literal'].isna() | df['feature_value_literal'].isna()]

print(f"Datos completos: {data_with_values.shape[0]} filas")
print(f"Datos faltantes: {data_missing_values.shape[0]} filas")

# Características y objetivos, items con literales asignados!
print("Preparando características y objetivos, items con los literales asignados ya!")
X_with_values = pd.get_dummies(data_with_values[['feature_category_id', 'feature_value_id']])
y_category_literal = data_with_values['feature_category_literal']
y_value_literal = data_with_values['feature_value_literal']

# Codificar literales
print("Codificando literales que ya tengo asignados...")
encoder_category_literal = LabelEncoder()
encoder_value_literal = LabelEncoder()

y_category_literal_encoded = encoder_category_literal.fit_transform(y_category_literal)
y_value_literal_encoded = encoder_value_literal.fit_transform(y_value_literal)

# Dividir el conjunto de datos
print("Dividiendo los datos en conjuntos de entrenamiento y prueba...")
X_train, X_test, y_train_category, y_test_category = train_test_split(X_with_values, y_category_literal_encoded, test_size=0.2, random_state=42)
X_train_value, X_test_value, y_train_value, y_test_value = train_test_split(X_with_values, y_value_literal_encoded, test_size=0.2, random_state=42)

# Entrenar modelos con validación cruzada
print("Entrenando modelo para 'feature_category_literal'...")
model_category_literal = RandomForestClassifier(n_estimators=100, random_state=42)
model_category_literal.fit(X_train, y_train_category)
cv_scores_category = cross_val_score(model_category_literal, X_with_values, y_category_literal_encoded, cv=5)
print(f"Validación cruzada para 'feature_category_literal': {cv_scores_category.mean()}")

#print("Entrenando modelo para 'feature_value_literal'...")
#model_value_literal = RandomForestClassifier(n_estimators=100, random_state=42)
#model_value_literal.fit(X_train_value, y_train_value)
#cv_scores_value = cross_val_score(model_value_literal, X_with_values, y_value_literal_encoded, cv=5)
#print(f"Validación cruzada para 'feature_value_literal': {cv_scores_value.mean()}")

# Preparar los datos faltantes para la imputación
print("Preparando datos faltantes para la imputación...")
X_missing = pd.get_dummies(df.loc[df['feature_category_literal'].isna() | df['feature_value_literal'].isna(), ['feature_category_id', 'feature_value_id']])

# Asegurarse de que las columnas están alineadas entre el conjunto de datos de entrenamiento y los datos faltantes
print("Alineando columnas...")
missing_cols = set(X_with_values.columns) - set(X_missing.columns)
for col in missing_cols:
    X_missing[col] = 0
X_missing = X_missing[X_with_values.columns]

# Imputar valores para feature_category_literal
print("Imputando valores para 'feature_category_literal'...")
predicted_category_literals = model_category_literal.predict(X_missing)
df.loc[df['feature_category_literal'].isna(), 'feature_category_literal'] = encoder_category_literal.inverse_transform(predicted_category_literals)

# Mostrar asignaciones después de la imputación
print("\nAsignaciones de 'feature_category_literal' después de la imputación:")
for idx, row in tqdm(df[df['feature_category_literal'].notna()][['feature_category_id', 'feature_category_literal']].drop_duplicates().iterrows(), total=df.shape[0], desc="Asignaciones feature_category_literal", unit="fila"):
    print(f"ID: {row['feature_category_id']}, Literal: {row['feature_category_literal']}")

# Imputar valores para feature_value_literal
#print("Imputando valores para 'feature_value_literal'...")
#predicted_value_literals = model_value_literal.predict(X_missing)
#df.loc[df['feature_value_literal'].isna(), 'feature_value_literal'] = encoder_value_literal.inverse_transform(predicted_value_literals)

# Mostrar asignaciones después de la imputación
#print("\nAsignaciones de 'feature_value_literal' después de la imputación:")
#for idx, row in tqdm(df[df['feature_value_literal'].notna()][['feature_value_id', 'feature_value_literal']].drop_duplicates().iterrows(), total=df.shape[0], desc="Asignaciones feature_value_literal", unit="fila"):
#    print(f"ID: {row['feature_value_id']}, Literal: {row['feature_value_literal']}")

# Verificar que los literales imputados están en el JSON
def validate_literals(df, column, literals_dict):
    print(f"Validando literales en '{column}'...")
    for category, literals in tqdm(literals_dict.items(), desc=f"Validando {column}", unit="categoría"):
        df.loc[df['feature_category_id'] == category, column] = df[column].apply(
            lambda x: x if x in literals else 'Desconocido'
        )

# Aplicar validación para feature_category_literal
validate_literals(df, 'feature_category_literal', category_literals)

# Aplicar validación para feature_value_literal
#validate_literals(df, 'feature_value_literal', category_literals)

# Guardar el CSV con los valores imputados
print("Guardando el DataFrame imputado en 'articulos_completados_modelo.csv'...")
df.to_csv('./item_features_ya_relleno_con_literales.csv', index=False)

print("Imputación completada y guardada en 'articulos_completados_modelo.csv'")


Preparando el DataFrame...
Separando datos con y sin valores faltantes...
Datos completos: 252104 filas
Datos faltantes: 219647 filas
Preparando características y objetivos, items con los literales asignados ya!
Codificando literales que ya tengo asignados...
Dividiendo los datos en conjuntos de entrenamiento y prueba...
Entrenando modelo para 'feature_category_literal'...
Validación cruzada para 'feature_category_literal': 1.0
Entrenando modelo para 'feature_value_literal'...


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Validación cruzada para 'feature_value_literal': 0.9999048013347606
Preparando datos faltantes para la imputación...
Alineando columnas...
Imputando valores para 'feature_category_literal'...

Asignaciones de 'feature_category_literal' después de la imputación:


Asignaciones feature_category_literal:   0%|                                   | 76/471751 [00:00<02:51, 2745.11fila/s]


ID: 56, Literal: rangos_de_precio
ID: 62, Literal: ocasion
ID: 68, Literal: productos
ID: 33, Literal: ocasion
ID: 72, Literal: estampados
ID: 29, Literal: materiales
ID: 16, Literal: ocasion
ID: 50, Literal: talla
ID: 61, Literal: estaciones
ID: 53, Literal: ocasion
ID: 7, Literal: caracteristicas
ID: 69, Literal: detalles
ID: 47, Literal: materiales
ID: 17, Literal: ocasion
ID: 32, Literal: generos
ID: 11, Literal: ocasion
ID: 45, Literal: ocasion
ID: 19, Literal: pais_de_fabricacion
ID: 46, Literal: ocasion
ID: 73, Literal: ocasion
ID: 55, Literal: marca
ID: 63, Literal: ocasion
ID: 59, Literal: ocasion
ID: 4, Literal: ocasion
ID: 5, Literal: ocasion
ID: 26, Literal: ocasion
ID: 3, Literal: ocasion
ID: 65, Literal: ocasion
ID: 18, Literal: ocasion
ID: 22, Literal: ocasion
ID: 41, Literal: ocasion
ID: 15, Literal: ocasion
ID: 44, Literal: ocasion
ID: 34, Literal: ocasion
ID: 12, Literal: ocasion
ID: 30, Literal: colores
ID: 21, Literal: ocasion
ID: 24, Literal: ocasion
ID: 28, Litera

Validando feature_category_literal: 100%|██████████████████████████████████████| 73/73 [00:00<00:00, 111.39categoría/s]


Guardando el DataFrame imputado en 'articulos_completados_modelo.csv'...
Imputación completada y guardada en 'articulos_completados_modelo.csv'


In [ ]:
# (1) Predicción con Restricción de Unicidad Usando RandomForestClassifier

import pandas as pd
import numpy as np
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from tqdm import tqdm
from scipy.optimize import linprog

# Cargar el fichero CSV
df = pd.read_csv('./item_features_semirelleno_con_literales_actualizado.csv')

# Cargar el fichero JSON
with open('./literales_ultimate.json', 'r', encoding='utf-8') as f:
    category_data = json.load(f)

# Convertir las columnas categóricas a texto temporalmente para evitar errores
df['feature_category_literal'] = df['feature_category_literal'].astype(str)
df['feature_value_literal'] = df['feature_value_literal'].astype(str)

# Preprocesar datos
def prepare_data(df, category_data):
    # Extraer literales de categorías y valores
    categories = list(category_data.keys())
    category_literals = {cat: literal for cat, literals in category_data.items() for literal in literals}
    
    # Crear mapeos para los literales y IDs
    category_id_to_literal = {i: cat for cat, i in enumerate(categories)}
    literal_to_id = {v: k for k, v in category_id_to_literal.items()}
    
    # Filtrar datos con valores no faltantes
    data_with_values = df.dropna(subset=['feature_category_literal', 'feature_value_literal'])
    data_missing_values = df[df['feature_category_literal'].isna() | df['feature_value_literal'].isna()]

    return data_with_values, data_missing_values, category_literals, category_id_to_literal, literal_to_id

# Crear modelos y transformadores
def create_model():
    return make_pipeline(
        ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(handle_unknown='ignore'), ['feature_category_id', 'feature_value_id'])
            ],
            remainder='passthrough'
        ),
        SimpleImputer(strategy='most_frequent'),
        RandomForestClassifier()
    )

def predict_with_unique_constraint(model, X_missing, encoder, assigned):
    # Verificar si X_missing está vacío
    if X_missing.empty:
        print("X_missing está vacío.")
        return np.array([])
    
    # Predicción con restricciones de unicidad
    predictions = model.predict(X_missing)
    predicted_values = encoder.inverse_transform(predictions)
    for i, value in enumerate(predicted_values):
        if value in assigned:
            predictions[i] = -1  # Marcar como no válido si ya asignado
    return encoder.inverse_transform(predictions)

# Preparar datos
data_with_values, data_missing_values, category_literals, category_id_to_literal, literal_to_id = prepare_data(df, category_data)

# Crear y ajustar modelos
model_category_literal = create_model()
model_value_literal = create_model()

# Entrenar modelos (esto es solo un ejemplo; usa tus datos de entrenamiento)
X_with_values = data_with_values[['feature_category_id', 'feature_value_id']]
y_category = data_with_values['feature_category_literal']
y_value = data_with_values['feature_value_literal']
model_category_literal.fit(X_with_values, y_category)
model_value_literal.fit(X_with_values, y_value)

# Preparar para predicción
X_missing = df[df['feature_category_literal'].isna() | df['feature_value_literal'].isna()][['feature_category_id', 'feature_value_id']]
assigned_categories = set(data_with_values['feature_category_literal'])
assigned_values = set(data_with_values['feature_value_literal'])

# Verificar si hay datos faltantes para imputar
if X_missing.empty:
    print("No hay datos faltantes para imputar.")
else:
    # Imputar valores faltantes
    df.loc[df['feature_category_literal'].isna(), 'feature_category_literal'] = predict_with_unique_constraint(model_category_literal, X_missing, category_id_to_literal, assigned_categories)
    df.loc[df['feature_value_literal'].isna(), 'feature_value_literal'] = predict_with_unique_constraint(model_value_literal, X_missing, literal_to_id, assigned_values)

# Guardar resultados
df.to_csv('./item_features_ya_relleno_con_literales.csv', index=False)
print("Asignaciones completadas y guardadas en './item_features_ya_relleno_con_literales.csv'")

In [1]:
# (2) Predicción con Múltiples Candidatos y Validación

import pandas as pd
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# Cargar el fichero CSV
df = pd.read_csv('./item_features_semirelleno_con_literales_actualizado.csv')

# Cargar el fichero JSON
with open('./literales_ultimate.json', 'r', encoding='utf-8') as f:
    category_data = json.load(f)

# Preparar el DataFrame
df['feature_category_id'] = df['feature_category_id'].astype('category')
df['feature_value_id'] = df['feature_value_id'].astype('category')
df['feature_category_literal'] = df['feature_category_literal'].astype('category')
df['feature_value_literal'] = df['feature_value_literal'].astype('category')

# Preparar los datos para la imputación
data_with_values = df.dropna(subset=['feature_category_literal', 'feature_value_literal'])
data_missing_values = df[df['feature_category_literal'].isna() | df['feature_value_literal'].isna()]

X_with_values = pd.get_dummies(data_with_values[['feature_category_id', 'feature_value_id']])
y_category_literal = data_with_values['feature_category_literal']
y_value_literal = data_with_values['feature_value_literal']

encoder_category_literal = LabelEncoder()
encoder_value_literal = LabelEncoder()

y_category_literal_encoded = encoder_category_literal.fit_transform(y_category_literal)
y_value_literal_encoded = encoder_value_literal.fit_transform(y_value_literal)

# Entrenar modelos
model_category_literal = RandomForestClassifier(n_estimators=100, random_state=42)
model_category_literal.fit(X_with_values, y_category_literal_encoded)

model_value_literal = RandomForestClassifier(n_estimators=100, random_state=42)
model_value_literal.fit(X_with_values, y_value_literal_encoded)

# Predicción con múltiples candidatos
def predict_multiple_candidates(model, X, encoder, n_candidates=5):
    probs = model.predict_proba(X)
    top_candidates = probs.argsort(axis=1)[:, -n_candidates:]  # n_candidates candidates
    decoded_candidates = []
    for idx, candidates in enumerate(top_candidates):
        decoded_values = encoder.inverse_transform(candidates)
        decoded_candidates.append(decoded_values)
    return decoded_candidates

# Imputar valores
X_missing = pd.get_dummies(df[df['feature_category_literal'].isna() | df['feature_value_literal'].isna()][['feature_category_id', 'feature_value_id']])
missing_cols = set(X_with_values.columns) - set(X_missing.columns)
for col in missing_cols:
    X_missing[col] = 0
X_missing = X_missing[X_with_values.columns]

# Imputar feature_category_literal
candidates_category_literals = predict_multiple_candidates(model_category_literal, X_missing, encoder_category_literal)
for i, (index, row) in enumerate(df[df['feature_category_literal'].isna()].iterrows()):
    for candidate in candidates_category_literals[i]:
        if candidate not in df['feature_category_literal'].values:
            df.at[index, 'feature_category_literal'] = candidate
            break

# Imputar feature_value_literal
candidates_value_literals = predict_multiple_candidates(model_value_literal, X_missing, encoder_value_literal)
for i, (index, row) in enumerate(df[df['feature_value_literal'].isna()].iterrows()):
    for candidate in candidates_value_literals[i]:
        if candidate not in df['feature_value_literal'].values:
            df.at[index, 'feature_value_literal'] = candidate
            break

df.to_csv('./item_features_ya_relleno_con_literales2.csv', index=False)

In [ ]:
# (3) Optimización de Imputación Usando Programación Matemática
import pandas as pd
import json
from ortools.linear_solver import pywraplp

# Cargar el fichero CSV
df = pd.read_csv('./item_features_semirelleno_con_literales.csv')

# Cargar el fichero JSON
with open('./literales.json', 'r', encoding='utf-8') as f:
    category_data = json.load(f)

# Preprocesar datos
def prepare_data(df, category_data):
    # Extraer literales de categorías y valores
    categories = list(category_data.keys())
    category_literals = {cat: literal for cat, literals in category_data.items() for literal in literals}
    
    # Crear mapeos para los literales y IDs
    category_id_to_literal = {i: cat for cat, i in enumerate(categories)}
    literal_to_id = {v: k for k, v in category_id_to_literal.items()}
    
    # Filtrar datos con valores no faltantes
    data_with_values = df.dropna(subset=['feature_category_literal', 'feature_value_literal'])
    data_missing_values = df[df['feature_category_literal'].isna() | df['feature_value_literal'].isna()]

    return data_with_values, data_missing_values, category_literals, category_id_to_literal, literal_to_id

def optimize_assignments(candidates, literals_to_assign):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    num_items = len(candidates)
    num_literals = len(literals_to_assign)

    # Variables de decisión: x[i, j] = 1 si la categoría i se asigna al literal j
    x = {}
    for i in range(num_items):
        for j in range(num_literals):
            x[i, j] = solver.BoolVar(f'x[{i},{j}]')
    
    # Restricción: cada ítem debe ser asignado exactamente una vez
    for i in range(num_items):
        solver.Add(sum(x[i, j] for j in range(num_literals)) == 1)

    # Restricción: cada literal debe ser asignado a un solo ítem
    for j in range(num_literals):
        solver.Add(sum(x[i, j] for i in range(num_items)) <= 1)

    # Objetivo: No es necesario un objetivo específico, solo resolver el problema
    solver.Minimize(0)
    
    # Resolver
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        assignment = {}
        for i in range(num_items):
            for j in range(num_literals):
                if x[i, j].solution_value() == 1:
                    assignment[i] = j
        return assignment
    else:
        print('No se encontró una solución óptima.')
        return None

# Preparar datos
data_with_values, data_missing_values, category_literals, category_id_to_literal, literal_to_id = prepare_data(df, category_data)

# Crear lista de candidatos y literales
candidates = data_missing_values.index.tolist()
literals_to_assign = list(set(category_id_to_literal.values()) - set(data_with_values['feature_category_literal'].unique()))

# Asignar literales
assignment = optimize_assignments(candidates, literals_to_assign)

# Aplicar asignaciones a los datos faltantes
if assignment:
    for idx, literal_idx in assignment.items():
        literal = literals_to_assign[literal_idx]
        df.at[candidates[idx], 'feature_category_literal'] = literal

df.to_csv('./item_features_ya_relleno_con_literales3.csv', index=False)